In [1]:
import requests,re,bs4
import sqlite3

In [2]:
url = "http://www.darklyrics.com/lyrics/haken/visions.html"
html = requests.get(url)
soup = bs4.BeautifulSoup(html.content,'html.parser')

In [3]:
metainfo    = [soup.find('h1').text,soup.find('h2').text]
metainfo[0] = metainfo[0].replace(' LYRICS','').capitalize()
metainfo.append(int(re.findall('\(([0-9]{4})\)',metainfo[1])[0]))
metainfo[1] = re.search('"(.*)"',metainfo[1]).groups()[0]
metainfo

['Haken', 'Visions', 2011]

In [4]:
lyrics     = soup.find('div', class_='lyrics')
tagnames   = [child.name for child in list(lyrics.children)]
begins     = [i for i,x in enumerate(tagnames) if x=='h3']
ends       = [i-1 for i in begins][1:] + [len(tagnames)]

In [5]:
class song(object):
    def __init__(self, name,lyrics,artist,genre,album,year):
        self.name   = name
        self.lyrics = lyrics
        self.artist = artist
        self.genre  = genre
        self.album  = album
        self.year   = year

In [6]:
songnames  = [list(lyrics.children)[i].text for i in begins]
songlyrics = [list(lyrics.children)[(x+1):(ends[i]+1)] for i,x in enumerate(begins)]
songlyrics = [[line for line in lyric if type(line) is bs4.element.NavigableString] for lyric in songlyrics]
songlyrics = [' '.join(lyric) for lyric in songlyrics]
print(songlyrics[1])


No disturbance could wake me 
Observers are captivated 
By nocturnal projections 
Of my vivid imagination 
 
Try as I might to recall the events of the night 
Yet the deeper I hunt, the further away they run 
 
Moon rises 
I shiver, confiding in the mirror 
My ghost in the reflection 
Just witnessed the resurrection 
 
Lights flashed before my eyes 
With scenes from someone else's life! 
 
Nothing is quite as it seems 
Something died inside of me 
By powers that be 
A man will live forever 
In paranoia 
 
I believe your conspiracy 
When they're after me 
For what I revealed could change your life 
 
Scenes from someone else's life 
Have they been placed in our minds? 
 
Never have I ever taken a life 
But it seems overnight I've become a killer 
On the run 'cause... 
 
I believe your conspiracy 
When they're after me 
For what I revealed 
 
I believe your conspiracy 
When they're after me 
For what I revealed will change your life 
 
Tonight I hang by a thread 
As the voice in my head

In [7]:
songs = []
for i,x in enumerate(songnames):
    songs.append(song(x,songlyrics[i],metainfo[0],'Progressive Metal',metainfo[1],metainfo[2]))

In [9]:
songs[2].__dict__.values()

dict_values(['3. Insomnia', "\nOver the edge \nFalling in and out of time \nHistory is always repeating \n \nDay in day out \nI wait to confront my fate \nMystery is never revealing \n \nI will wait \nI will wait \n \nNight after night \nShrouded in secrecy \nThe harder I try to decipher this symphony \nIt pushes me closer to death \nI begin to see alternate realities surrounding me \n \nI am dying to get back to sleep again \nI lie in my bed but I've seen the end \nMy days are numbered \nIn death I discover \n \nInsomnia \nInsomnia \nI will wait \nI will wait \n \nDying to get back to sleep again \nI lie in my bed but I've seen the end \nMy days are numbered \nIn death I discover \n \nInsomnia \nInsomnia \n \n", 'Haken', 'Progressive Metal', 'Visions', 2011])

In [12]:
songs[2].__dict__['album']

'Visions'

## JSON'a yazma

In [19]:
import json, os

with open('output/%s_%s.json' % (songs[0].__dict__['artist'].replace(' ',''),songs[0].__dict__['album'].replace(' ','')), 'w') as fh:
    fh.write('[')
    for song in songs:
        fh.write('\n')
        json.dump(song.__dict__, fh)
        fh.write(',')
    # Son iterasyonda eklenen gereksiz virgül'ü sil
    fh.seek(fh.tell()-1)
    fh.truncate()
    # Bir satır aşağıya
    fh.write('\n]')

In [21]:
import json
fh = open('output/Haken_Visions.json')
print(json.load(fh)[1])
fh.close()

{'name': '2. Nocturnal Conspiracy', 'lyrics': "\nNo disturbance could wake me \nObservers are captivated \nBy nocturnal projections \nOf my vivid imagination \n \nTry as I might to recall the events of the night \nYet the deeper I hunt, the further away they run \n \nMoon rises \nI shiver, confiding in the mirror \nMy ghost in the reflection \nJust witnessed the resurrection \n \nLights flashed before my eyes \nWith scenes from someone else's life! \n \nNothing is quite as it seems \nSomething died inside of me \nBy powers that be \nA man will live forever \nIn paranoia \n \nI believe your conspiracy \nWhen they're after me \nFor what I revealed could change your life \n \nScenes from someone else's life \nHave they been placed in our minds? \n \nNever have I ever taken a life \nBut it seems overnight I've become a killer \nOn the run 'cause... \n \nI believe your conspiracy \nWhen they're after me \nFor what I revealed \n \nI believe your conspiracy \nWhen they're after me \nFor what 

## SQL'e yazma

In [217]:
import sqlite3
conn = sqlite3.connect('outputs/lyrics.db')

conn.execute('DROP TABLE lyrics') # daha önce oluşturduğumuz tabloyu sil
cur  = conn.cursor()

cur.execute('''CREATE TABLE lyrics
             (name TEXT, lyrics TEXT, artist TEXT, genre TEXT, album TEXT,year INTEGER)''')
cur.execute('''CREATE UNIQUE INDEX id ON lyrics (name, artist, album)''')

for s in songs:
    # Yazma
    cur.execute("INSERT INTO lyrics VALUES (?,?,?,?,?,?)", (s.name,s.lyrics,s.artist,s.genre,s.album,s.year))

conn.commit()    
conn.close()

In [219]:
import sqlite3
conn = sqlite3.connect('outputs/lyrics.db')
cur  = conn.cursor()

for s in songs:
    # Yazma
    cur.execute("INSERT OR IGNORE INTO lyrics VALUES (?,?,?,?,?,?)", (s.name,s.lyrics,s.artist,s.genre,s.album,s.year))

conn.commit()    
conn.close()

## Album URL üretme

In [176]:
url  = 'http://www.darklyrics.com/d/dreamtheater.html'
soup = bs4.BeautifulSoup(requests.get(url).content,'html.parser')
soup.prettify()

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">\n <head>\n  <title>\n   DREAM THEATER lyrics\n  </title>\n  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>\n  <meta content=\'DREAM THEATER lyrics - 154 song lyrics from 16 albums, including "The Astonishing" (2016).\' name="description"/>\n  <meta content="DREAM THEATER, lyrics, discography, albums, songs" name="keywords"/>\n  <link href="../dark.css" rel="stylesheet" type="text/css"/>\n  <script type="text/javascript">\n   var _gaq = _gaq || [];\r\n  _gaq.push([\'_setAccount\', \'UA-20396473-1\']);\r\n  _gaq.push([\'_trackPageview\']);\r\n\r\n  (function() {\r\n    var ga = document.createElement(\'script\'); ga.type = \'text/javascript\'; ga.async = true;\r\n    ga.src = (\'https:\' == document.location.protocol ? \'https://ssl\' : \'http://www\') + \'.google-analytics.com

In [187]:
url_list = [album.find('a').get('href') for album in soup.find_all('div', class_='album')]
url_list = [re.sub('#1|\.\./','',url) for url in url_list]
url_list = ['http://www.darklyrics.com/' + url for url in url_list]
url_list

['http://www.darklyrics.com/lyrics/dreamtheater/whendreamanddayunite.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/imagesandwords.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/awake.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/achangeofseasons.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/fallingintoinfinity.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/scenesfromamemory.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/sixdegreesofinnerturbulence.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/trainofthought.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/octavarium.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/score.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/systematicchaos.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/blackcloudssilverlinings.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/adramaticturnofevents.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/dreamtheater.html

# Darklyrics Crawler
INPUT  : Album URL, genre
OUTPUT : Artist name, album name, genre, release year, lyrics, song name (WRITTEN to SQL)

In [189]:
import requests,re,bs4
import sqlite3

class song(object):
    def __init__(self, name,lyrics,artist,genre,album,year):
        self.name   = name
        self.lyrics = lyrics
        self.artist = artist
        self.genre  = genre
        self.album  = album
        self.year   = year

In [190]:
def album_scraper(url,genre):
    html = requests.get(url)
    soup = bs4.BeautifulSoup(html.content,'html.parser')
    # Meta info extraction
    metainfo    = [soup.find('h1').text,soup.find('h2').text]
    metainfo[0] = metainfo[0].replace(' LYRICS','').capitalize()
    metainfo.append(int(re.findall('\(([0-9]{4})\)',metainfo[1])[0]))
    metainfo[1] = re.search('"(.*)"',metainfo[1]).groups()[0]
    # lyric extraction
    lyrics     = soup.find('div', class_='lyrics')
    tagnames   = [child.name for child in list(lyrics.children)]
    begins     = [i for i,x in enumerate(tagnames) if x=='h3']
    ends       = [i-1 for i in begins][1:] + [len(tagnames)]
    # song name and lyrics
    songnames  = [list(lyrics.children)[i].text for i in begins]
    songlyrics = [list(lyrics.children)[(x+1):(ends[i]+1)] for i,x in enumerate(begins)]
    songlyrics = [[line.strip() for line in lyric if type(line) is bs4.element.NavigableString] for lyric in songlyrics]
    songlyrics = [' '.join(lyric) for lyric in songlyrics]
    #
    # Creation of song and Album (songs list)
    songs = []
    for i,x in enumerate(songnames):
        songs.append(song(x,songlyrics[i],metainfo[0],genre,metainfo[1],metainfo[2]))
        print('Scraped %s of %s, %s' % (x,metainfo[0],metainfo[1]))
    return(songs)

In [215]:
def albumwriter(song_info_list,dbdest):
    conn = sqlite3.connect(dbdest)
    cur  = conn.cursor()

    for s in song_info_list:
        # Yazma
        cur.execute("INSERT INTO lyrics VALUES (?,?,?,?,?,?)", (s.name,s.lyrics,s.artist,s.genre,s.album,s.year))
        print('[LOG] Written songs for %s - %s' % (s.album, s.artist))
    conn.commit()    
    conn.close()

In [ ]:
import json, os
def jsonwriter(song_info_list,dbdest):
    filename = 'output/%s_%s.json' % (songs[0].__dict__['artist'].replace(' ',''),songs[0].__dict__['album'].replace(' ',''))
    with open(filename, 'w') as fh:
        fh.write('[')
        for song in songs:
            fh.write('\n')
            json.dump(song.__dict__, fh)
            fh.write(',')
        # Son iterasyonda eklenen gereksiz virgül'ü sil
        fh.seek(fh.tell()-1)
        fh.truncate()
        # Bir satır aşağıya
        fh.write('\n]')
    

In [206]:
def url_extractor(band_name,genre):
    band_name = band_name.replace(' ','').lower()
    print(band_name)
    first_letter = band_name[0]
    url  = 'http://www.darklyrics.com/%s/%s.html' % (first_letter,band_name)
    soup = bs4.BeautifulSoup(requests.get(url).content,'html.parser')
    url_list = [album.find('a').get('href') for album in soup.find_all('div', class_='album')]
    url_list = [re.sub('#1|\.\./','',url) for url in url_list]
    url_list = ['http://www.darklyrics.com/' + url for url in url_list]
    print('[LOG]: Scraped %i urls for %s' % (len(url_list),band_name))
    return(url_list)

In [199]:
song_list = dlscraper('http://www.darklyrics.com/lyrics/dreamtheater/imagesandwords.html','Progressive Metal')

Scraped 1. Pull Me Under in Dream theater, Images And Words
Scraped 2. Another Day in Dream theater, Images And Words
Scraped 3. Take The Time in Dream theater, Images And Words
Scraped 4. Surrounded in Dream theater, Images And Words
Scraped 5. Metropolis - Part I - "The Miracle And The Sleeper" in Dream theater, Images And Words
Scraped 6. Under A Glass Moon in Dream theater, Images And Words
Scraped 7. Wait For Sleep in Dream theater, Images And Words
Scraped 8. Learning To Live in Dream theater, Images And Words


In [207]:
url_list = url_extractor('Dream Theater','Progressive Metal')
url_list

dreamtheater
[LOG]: Scraped 16 urls for dreamtheater


['http://www.darklyrics.com/lyrics/dreamtheater/whendreamanddayunite.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/imagesandwords.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/awake.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/achangeofseasons.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/fallingintoinfinity.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/scenesfromamemory.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/sixdegreesofinnerturbulence.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/trainofthought.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/octavarium.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/score.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/systematicchaos.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/blackcloudssilverlinings.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/adramaticturnofevents.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/dreamtheater.html

In [210]:
song_lists = [dlscraper(url,'Progressive Metal') for url in url_list]

Scraped 1. A Fortune In Lies in Dream theater, When Dream And Day Unite
Scraped 2. Status Seeker in Dream theater, When Dream And Day Unite
Scraped 3. The Ytse Jam in Dream theater, When Dream And Day Unite
Scraped 4. The Killing Hand in Dream theater, When Dream And Day Unite
Scraped 5. Light Fuse And Get Away in Dream theater, When Dream And Day Unite
Scraped 6. Afterlife in Dream theater, When Dream And Day Unite
Scraped 7. The Ones Who Help To Set The Sun in Dream theater, When Dream And Day Unite
Scraped 8. Only A Matter Of Time in Dream theater, When Dream And Day Unite
Scraped 1. Pull Me Under in Dream theater, Images And Words
Scraped 2. Another Day in Dream theater, Images And Words
Scraped 3. Take The Time in Dream theater, Images And Words
Scraped 4. Surrounded in Dream theater, Images And Words
Scraped 5. Metropolis - Part I - "The Miracle And The Sleeper" in Dream theater, Images And Words
Scraped 6. Under A Glass Moon in Dream theater, Images And Words
Scraped 7. Wait For

Scraped 1. Descent Of The Nomacs in Dream theater, The Astonishing
Scraped 2. Dystopian Overture in Dream theater, The Astonishing
Scraped 3. The Gift Of Music in Dream theater, The Astonishing
Scraped 4. The Answer in Dream theater, The Astonishing
Scraped 5. A Better Life in Dream theater, The Astonishing
Scraped 6. Lord Nafaryus in Dream theater, The Astonishing
Scraped 7. A Savior In The Square in Dream theater, The Astonishing
Scraped 8. When Your Time Has Come in Dream theater, The Astonishing
Scraped 9. Act Of Faythe in Dream theater, The Astonishing
Scraped 10. Three Days in Dream theater, The Astonishing
Scraped 11. The Hovering Sojourn in Dream theater, The Astonishing
Scraped 12. Brother, Can You Hear Me? in Dream theater, The Astonishing
Scraped 13. A Life Left Behind in Dream theater, The Astonishing
Scraped 14. Ravenskill in Dream theater, The Astonishing
Scraped 15. Chosen in Dream theater, The Astonishing
Scraped 16. A Tempting Offer in Dream theater, The Astonishing
Sc

In [220]:
for song_list in song_lists:
    albumwriter(song_list, 'outputs/lyrics.db')

[LOG] Written songs for When Dream And Day Unite - Dream theater
[LOG] Written songs for When Dream And Day Unite - Dream theater
[LOG] Written songs for When Dream And Day Unite - Dream theater
[LOG] Written songs for When Dream And Day Unite - Dream theater
[LOG] Written songs for When Dream And Day Unite - Dream theater
[LOG] Written songs for When Dream And Day Unite - Dream theater
[LOG] Written songs for When Dream And Day Unite - Dream theater
[LOG] Written songs for When Dream And Day Unite - Dream theater
[LOG] Written songs for Images And Words - Dream theater
[LOG] Written songs for Images And Words - Dream theater
[LOG] Written songs for Images And Words - Dream theater
[LOG] Written songs for Images And Words - Dream theater
[LOG] Written songs for Images And Words - Dream theater
[LOG] Written songs for Images And Words - Dream theater
[LOG] Written songs for Images And Words - Dream theater
[LOG] Written songs for Images And Words - Dream theater
[LOG] Written songs for 

In [214]:
song_list[0].artist

'Dream theater'

In [221]:
song_list = dlscraper('http://www.darklyrics.com/lyrics/haken/visions.html','Progressive Metal')

Scraped 1. Premonition in Haken, Visions
Scraped 2. Nocturnal Conspiracy in Haken, Visions
Scraped 3. Insomnia in Haken, Visions
Scraped 4. The Mind's Eye in Haken, Visions
Scraped 5. Portals in Haken, Visions
Scraped 6. Shapeshifter in Haken, Visions
Scraped 7. Deathless in Haken, Visions
Scraped 8. Visions in Haken, Visions


In [227]:
song_list[4].lyrics

' '

# Lyrics Crawler (Object) 

In [103]:
tx = "I'm an album name"

error: unterminated character set at position 3

In [9]:
import requests,re,bs4
import sqlite3
from time import sleep
from random import uniform
import json, os


class DarkLyricsCrawler(object):
    
    class song(object):
        def __init__(self, name,lyrics,artist,genre,album,year,source):
            self.name   = name
            self.lyrics = lyrics
            self.artist = artist
            self.genre  = genre
            self.album  = album
            self.year   = year
            self.source = source
            
    def url_extractor(self,band_name,genre):
        band_name = band_name.replace(' ','').lower()
        first_letter = band_name[0]
        url  = 'http://www.darklyrics.com/%s/%s.html' % (first_letter,band_name)
        soup = bs4.BeautifulSoup(requests.get(url).content,'html.parser')
        url_list = [album.find('a').get('href') for album in soup.find_all('div', class_='album')]
        url_list = [re.sub('#1|\.\./','',url) for url in url_list]
        url_list = ['http://www.darklyrics.com/' + url for url in url_list]
        print('[LOG]: Scraped %i urls for %s' % (len(url_list),band_name))
        sleep(uniform(1,5))
        return((url_list,genre))

    def album_scraper(self,url,genre):
        html = requests.get(url)
        soup = bs4.BeautifulSoup(html.content,'html.parser')
        # Meta info extraction
        metainfo    = [soup.find('h1').text,soup.find('h2').text]
        metainfo[0] = metainfo[0].replace(' LYRICS','').capitalize()
        metainfo.append(''.join(re.findall('\(([0-9]{4})\)',metainfo[1])))
        if not metainfo[2]:
            metainfo[2] == None
            metainfo[1] == None
        else:
            metainfo[2] = int(metainfo[2])
            metainfo[1] = re.search('"(.*)"',metainfo[1]).groups()[0]
        # lyric extraction
        lyrics     = soup.find('div', class_='lyrics')
        tagnames   = [child.name for child in list(lyrics.children)]
        begins     = [i for i,x in enumerate(tagnames) if x=='h3']
        ends       = [i-1 for i in begins][1:] + [len(tagnames)]
        # song name and lyrics
        songnames  = [list(lyrics.children)[i].text for i in begins]
        songlyrics = [list(lyrics.children)[(x+1):(ends[i]+1)] for i,x in enumerate(begins)]
        songlyrics = [[line.strip() for line in lyric if type(line) is bs4.element.NavigableString] for lyric in songlyrics]
        songlyrics = [' '.join(lyric) for lyric in songlyrics]
        #
        # Creation of song and Album (songs list)
        songs = []
        for i,x in enumerate(songnames):
            songs.append(self.song(x,songlyrics[i],metainfo[0],genre,metainfo[1],metainfo[2],'Darklyrics'))
            print('Scraped %s of %s, %s' % (x,metainfo[0],metainfo[1]))
        sleep(uniform(4,20))
        return(songs)

    def db_creator(self, dbpath, dbname):
        conn = sqlite3.connect(dbpath + '/' + dbname)
        cur  = conn.cursor()
        cur.execute('''CREATE TABLE lyrics
                     (name TEXT, lyrics TEXT, artist TEXT, genre TEXT, album TEXT,year INTEGER, source TEXT)''')
        cur.execute('''CREATE UNIQUE INDEX id ON lyrics (name, artist, album)''')

        conn.commit()    
        conn.close()

    def jsonwriter(self,song_info_list,dbpath):
        if not os.path.isdir(dbpath + '/json'): os.mkdir(dbpath + '/json')
        filename = '%s/%s_%s.json' % (dbpath + '/json',
                                      re.sub('[\s\'\.,~^+%&/)(=\?]','',song_info_list[0].artist),
                                      re.sub('[\s\'\.,~^+%&/)(=\?]','',song_info_list[0].album))
        with open(filename, 'w') as fh:
            fh.write('[')
            for song in song_info_list:
                fh.write('\n')
                json.dump(song.__dict__, fh)
                fh.write(',')
            # Son iterasyonda eklenen gereksiz virgül'ü sil
            fh.seek(fh.tell()-1)
            fh.truncate()
            # Bir satır aşağıya
            fh.write('\n]')
        print('[LOG] Written songs for %s - %s to JSON' % (song_info_list[0].album, song_info_list[0].artist))
    
    def album_writer(self, song_info_list,dbdest):
        conn = sqlite3.connect(dbdest)
        cur  = conn.cursor()

        for s in song_info_list:
            # Yazma
            cur.execute("INSERT OR IGNORE INTO lyrics VALUES (?,?,?,?,?,?,?)", 
                        (s.name,s.lyrics,s.artist,s.genre,s.album,s.year,s.source))
            print('[LOG] Written songs for %s - %s' % (s.album, s.artist))
        conn.commit()    
        conn.close()

In [12]:
dc = DarkLyricsCrawler()

In [13]:
with open('bands.txt') as bands:
    band_list = [tuple(band.strip().split(',')) for band in bands]
print(band_list)

[('Cannibal Corpse', 'Death Metal'), ('Suffocation', 'Death Metal'), ('Deicide', 'Death Metal'), ('Obituary', 'Death Metal'), ('Dying Fetus', 'Death Metal')]


In [14]:
#band_list  = [('Slipknot','Nu Metal'),('Pantera','Power Metal'), ('Deftones','Nu Metal')]
album_urls = [dc.url_extractor(band,genre) for band,genre in band_list]

[LOG]: Scraped 16 urls for cannibalcorpse
[LOG]: Scraped 10 urls for suffocation
[LOG]: Scraped 11 urls for deicide
[LOG]: Scraped 10 urls for obituary
[LOG]: Scraped 11 urls for dyingfetus


In [115]:
album_urls

[(['http://www.darklyrics.com/lyrics/queensryche/queensrche.html',
   'http://www.darklyrics.com/lyrics/queensryche/thewarning.html',
   'http://www.darklyrics.com/lyrics/queensryche/ragefororder.html',
   'http://www.darklyrics.com/lyrics/queensryche/operationmindcrime.html',
   'http://www.darklyrics.com/lyrics/queensryche/empire.html',
   'http://www.darklyrics.com/lyrics/queensryche/promisedland.html',
   'http://www.darklyrics.com/lyrics/queensryche/hearinthenowfrontier.html',
   'http://www.darklyrics.com/lyrics/queensryche/q2k.html',
   'http://www.darklyrics.com/lyrics/queensryche/liveevolution.html',
   'http://www.darklyrics.com/lyrics/queensryche/tribe.html',
   'http://www.darklyrics.com/lyrics/queensryche/operationmindcrimeii.html',
   'http://www.darklyrics.com/lyrics/queensryche/americansoldier.html',
   'http://www.darklyrics.com/lyrics/queensryche/dedicatedtochaos.html',
   'http://www.darklyrics.com/lyrics/queensryche/frequencyunknown.html',
   'http://www.darklyrics.

In [15]:
albums = []
for urls, genre in album_urls:
    for url in urls:
        album = dc.album_scraper(url, genre)
        albums.append(album)
        dc.jsonwriter(album,'output')

Scraped 1. Shredded Humans of Cannibal corpse, Eaten Back To Life
Scraped 2. Edible Autopsy of Cannibal corpse, Eaten Back To Life
Scraped 3. Put Them To Death of Cannibal corpse, Eaten Back To Life
Scraped 4. Mangled of Cannibal corpse, Eaten Back To Life
Scraped 5. Scattered Remains, Splattered Brains of Cannibal corpse, Eaten Back To Life
Scraped 6. Born In A Casket of Cannibal corpse, Eaten Back To Life
Scraped 7. Rotting Head of Cannibal corpse, Eaten Back To Life
Scraped 8. The Undead Will Feast of Cannibal corpse, Eaten Back To Life
Scraped 9. Bloody Chunks of Cannibal corpse, Eaten Back To Life
Scraped 10. A Skull Full Of Maggots of Cannibal corpse, Eaten Back To Life
Scraped 11. Buried In The Backyard of Cannibal corpse, Eaten Back To Life
[LOG] Written songs for Eaten Back To Life - Cannibal corpse to JSON
Scraped 1. Meat Hook Sodomy of Cannibal corpse, Butchered At Birth
Scraped 2. Gutted of Cannibal corpse, Butchered At Birth
Scraped 3. Living Dissection of Cannibal corpse,

[LOG] Written songs for Kill - Cannibal corpse to JSON
Scraped 1. Priest Of Sodom of Cannibal corpse, Evisceration Plague
Scraped 2. Scalding Hail of Cannibal corpse, Evisceration Plague
Scraped 3. To Decompose of Cannibal corpse, Evisceration Plague
Scraped 4. A Cauldron Of Hate of Cannibal corpse, Evisceration Plague
Scraped 5. Beheading And Burning of Cannibal corpse, Evisceration Plague
Scraped 6. Evidence In The Furnace of Cannibal corpse, Evisceration Plague
Scraped 7. Carnivorous Swarm of Cannibal corpse, Evisceration Plague
Scraped 8. Evisceration Plague of Cannibal corpse, Evisceration Plague
Scraped 9. Shatter Their Bones of Cannibal corpse, Evisceration Plague
Scraped 10. Carrion Sculpted Entity of Cannibal corpse, Evisceration Plague
Scraped 11. Unnatural of Cannibal corpse, Evisceration Plague
Scraped 12. Skewered From Ear To Eye of Cannibal corpse, Evisceration Plague
Scraped 13. Skull Fragment Armor of Cannibal corpse, Evisceration Plague
[LOG] Written songs for Eviscera

[LOG] Written songs for Pinnacle Of Bedlam - Suffocation to JSON
Scraped 1. Clarity Through Deprivation of Suffocation, ...Of The Dark Light
Scraped 2. The Warmth Within The Dark of Suffocation, ...Of The Dark Light
Scraped 3. Your Last Breaths of Suffocation, ...Of The Dark Light
Scraped 4. Return To The Abyss of Suffocation, ...Of The Dark Light
Scraped 5. The Violation of Suffocation, ...Of The Dark Light
Scraped 6. Of The Dark Light of Suffocation, ...Of The Dark Light
Scraped 7. Some Things Should Be Left Alone of Suffocation, ...Of The Dark Light
Scraped 8. Caught Between Two Worlds of Suffocation, ...Of The Dark Light
Scraped 9. Epitaph Of The Credulous of Suffocation, ...Of The Dark Light
[LOG] Written songs for ...Of The Dark Light - Suffocation to JSON
Scraped 1. Lunatic Of God's Creation of Deicide, Deicide
Scraped 2. Sacrificial Suicide of Deicide, Deicide
Scraped 3. Oblivious To Evil of Deicide, Deicide
Scraped 4. Dead By Dawn of Deicide, Deicide
Scraped 5. Blaspherereion 

[LOG] Written songs for Slowly We Rot - Obituary to JSON
Scraped 1. Infected of Obituary, Cause Of Death
Scraped 2. Body Bag of Obituary, Cause Of Death
Scraped 3. Chopped In Half of Obituary, Cause Of Death
Scraped 4. Circle Of The Tyrants of Obituary, Cause Of Death
Scraped 5. Dying of Obituary, Cause Of Death
Scraped 6. Find The Arise of Obituary, Cause Of Death
Scraped 7. Cause Of Death of Obituary, Cause Of Death
Scraped 8. Memories Remain of Obituary, Cause Of Death
Scraped 9. Turned Inside Out of Obituary, Cause Of Death
Scraped 10. Infected of Obituary, Cause Of Death
Scraped 11. Memories Remain of Obituary, Cause Of Death
Scraped 12. Chopped In Half of Obituary, Cause Of Death
[LOG] Written songs for Cause Of Death - Obituary to JSON
Scraped 1. I'm In Pain of Obituary, The End Complete
Scraped 2. Back To One of Obituary, The End Complete
Scraped 3. Dead Silence of Obituary, The End Complete
Scraped 4. In The End Of Life of Obituary, The End Complete
Scraped 5. Sickness of Obit

Scraped 1. Praise The Lord (Opium Of The Masses) of Dying fetus, Destroy The Opposition
Scraped 2. Destroy The Opposition of Dying fetus, Destroy The Opposition
Scraped 3. Born In Sodom of Dying fetus, Destroy The Opposition
Scraped 4. Epidemic Of Hate of Dying fetus, Destroy The Opposition
Scraped 5. Pissing In The Mainstream of Dying fetus, Destroy The Opposition
Scraped 6. In Times Of War of Dying fetus, Destroy The Opposition
Scraped 7. For Us Or Against Us of Dying fetus, Destroy The Opposition
Scraped 8. Justifiable Homicide of Dying fetus, Destroy The Opposition
[LOG] Written songs for Destroy The Opposition - Dying fetus to JSON
Scraped 1. Grotesque Impalement of Dying fetus, Grotesque Impalement
Scraped 2. Streaks Of Blood of Dying fetus, Grotesque Impalement
Scraped 3. Bringing Back The Glory of Dying fetus, Grotesque Impalement
Scraped 4. Tearing Inside The Womb of Dying fetus, Grotesque Impalement
Scraped 5. Final Evil (Prelude To Evil: Davey's Nightmare) of Dying fetus, Gr

In [20]:
dc.album_writer(albums[1],'output/lyrics.db')

[LOG] Written songs for Aeolia - Leprous
[LOG] Written songs for Aeolia - Leprous
[LOG] Written songs for Aeolia - Leprous
[LOG] Written songs for Aeolia - Leprous
[LOG] Written songs for Aeolia - Leprous
[LOG] Written songs for Aeolia - Leprous
[LOG] Written songs for Aeolia - Leprous
[LOG] Written songs for Aeolia - Leprous
[LOG] Written songs for Aeolia - Leprous


# azlyrics.com 

In [9]:
import requests,re,bs4
import sqlite3
from time import sleep
from random import uniform, randint

url = 'https://raw.githubusercontent.com/asuerdem/SMM/master/1_Crawler/1_newscrawler/user_agent_list.txt'
useragents = requests.get(url)

In [10]:
useragents = useragents.text.split('\n')

In [14]:
from ghost import Ghost
ghost = Ghost()

url = 'https://www.azlyrics.com/f/faithnomore.html'

with ghost.start() as session:
    page, extra_resources = session.open(url)
    assert page.http_status == 200 and 'jeanphix' in page.content


AttributeError: 'NoneType' object has no attribute 'http_status'

In [13]:
useragent = useragents[randint(0,len(useragents) - 1)]
headers = { 'User-Agent': useragent }
url = 'https://www.azlyrics.com/a/adele.html'
html = requests.get(url, headers=headers)
soup = bs4.BeautifulSoup(html.content,'html.parser')

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))

In [6]:
artist_info = soup.find('h1').text
artist_info = artist_info.replace(' Lyrics', '')
artist_info

'Adele'

In [7]:
albums_info = []
for info in soup.find_all('div', class_='album'):
    album_name  = info.text
    release_year = ''.join(re.findall('\(([0-9]+)\)',album_name))
    release_year = int(release_year) if release_year else None
    album_name = ''.join(re.findall(':\s(.*)\s\(',album_name))
    if not album_name: album_name = 'Other Songs' 
    albums_info.append({'Album': album_name, 'Year': release_year})
albums_info

[{'Album': '"19"', 'Year': 2008},
 {'Album': '"21"', 'Year': 2011},
 {'Album': '"25"', 'Year': 2015},
 {'Album': 'Other Songs', 'Year': None}]

In [115]:
album_lyrics = soup.find('div', attrs = {'id':'listAlbum'})
# album_lyrics = album_lyrics.find('a', attrs = {'id':'listAlbum'}0)
album_lyrics = [part for part in list(album_lyrics.children) if part!= '\n']
begins = [i for i, part in enumerate(album_lyrics) if part.name == 'div' and part.has_attr('class')]
ends   = [i-1 for i in begins][1:] + [len(album_lyrics)]
albums = [album_lyrics[begins[i]:ends[i]] for i in range(len(begins))]

In [230]:
def extract_lyrics(url):    
    headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0' }
    html = requests.get(url, headers=headers)
    soup = bs4.BeautifulSoup(html.content,'html.parser')

    lyrics = soup.select('div[class*="text-center"]')
    ind = [len(part.find_all('br')) for part in lyrics]
    ind = [i for i,v in enumerate(ind) if v == max(ind)][0]
    # aa = lyrics[ind].findChild('div', attrs={'class':'ringtone'})
    lyrics = lyrics[ind].findChildren('div')
    ind = [len(part.find_all('br')) for part in lyrics]
    ind = [i for i,v in enumerate(ind) if v == max(ind)][0]
    return(lyrics[ind].text)

In [241]:
artist_info = soup.find('h1').text
artist_info = artist_info.replace(' Lyrics', '')
genre       = 'Pop'
# albums_info = []
songs       = []
for album in albums:
    release_year = ''.join(re.findall('\(([0-9]+)\)',album[0].text))
    release_year = int(release_year) if release_year else None
    album_name = ''.join(re.findall(':\s(.*)\s\(',album[0].text))
    if not album_name: album_name = 'Other Songs' 
    # albums_info.append({'Album': album_name, 'Year': release_year})
    for song in album:
        if song.name == 'a':
            songname = song.text
            songlink = song.attrs['href']
            songlink = 'https://www.azlyrics.com/' + songlink.replace('../','')
            lyrics   = extract_lyrics(songlink)
            sleep(uniform(.5,1))
            songs.append({'Album': album_name, 'Year': release_year, 'Name': songname,
                          'Link': songlink, 'Artist':artist_info,'Lyrics':lyrics,'Genre':genre})
    sleep(uniform(.5,1))

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))

In [234]:
from time import sleep
for song in songs[:4]:
    sleep(.4)
    print(song['Lyrics'])



Daydreamer
Sitting on the sea
Soaking up the sun
He is a real lover
Of making up the past
And feeling up his girl
Like he's never felt a figure before

A jaw-dropper
Looks good when he walks
Is the subject of their talk
He would be hard to chase
But good to catch
And he could change the world
With his hands behind his back, oh

You can find him sittin' on your doorstep
Waiting for a surprise
And he will feel like he's been there for hours
And you can tell that he'll be there for life

Daydreamer
With eyes that make you melt
He lends his coat for shelter
Plus he's there for you
When he shouldn't be
But he stays all the same
Waits for you
Then sees you through

There's no way I could describe him
What I've said is just what I'm hoping for

But I will find him sittin' on my doorstep
Waiting for a surprise
And he will feel like he's been there for hours
And I can tell that he'll be there for life
And I can tell that he'll be there for life



Wait, do you see my heart on my sleeve?
It's 

In [238]:
def db_creator(dbpath, dbname):
    conn = sqlite3.connect(dbpath + '/' + dbname)
    cur  = conn.cursor()
    cur.execute('''CREATE TABLE lyrics
                 (name TEXT, lyrics TEXT, artist TEXT, genre TEXT, album TEXT,year INTEGER, source TEXT)''')
    cur.execute('''CREATE UNIQUE INDEX id ON lyrics (name, artist, album)''')

    conn.commit()    
    conn.close()

def album_writer(song_info_list,dbdest):
    conn = sqlite3.connect(dbdest)
    cur  = conn.cursor()

    for s in song_info_list:
        # Yazma
        cur.execute("INSERT OR IGNORE INTO lyrics VALUES (?,?,?,?,?,?,?)", 
                    (s['Name'],s['Lyrics'],s['Artist'],s['Genre'],s['Album'],s['Year'],'azlyrics'))
        print('[LOG] Written songs for %s - %s' % (s.album, s.artist))
    conn.commit()    
    conn.close()

In [239]:
album_writer(songs,'output/lyrics.db')

In [240]:
songs

[]